In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import json

import pyalex
pyalex.config.email = "t.kagel@uu.nl"

client_orcid = open('orcid_config.json')
client_orcid = json.load(client_orcid)

client_scopus = open('scopus_config.json')
client_scopus = json.load(client_scopus)

df_names = pd.read_csv('people_in_S4S_pureFiltered.csv',index_col="Index")
df_names.reset_index(inplace=True)
df_names.drop('Index',axis=1,inplace=True)
df_names.index.names = ['Index']

#orcid start up
client_id = client_orcid['client_id']
client_secret = client_orcid['client_secret']

resp = requests.post(url="https://orcid.org/oauth/token",
                     headers={'Accept':'application/json'},
                     data={'client_id':client_id, 'client_secret':client_secret,'grant_type': 'client_credentials', 'scope':'/read-public'}
                     )
access_token = resp.json()['access_token']


In [2]:
col_names = ['solisID','pureID','lastname','firstname', 'scopusID','orcID', 'jobtitle','organisationalUnit','numberResearchOutputs','sampleTitle','sampleDOI','sampleType']
df_pure = pd.read_excel('pure_data.xlsx',names=col_names,keep_default_na='')
df_pure['firstname'] = df_pure['firstname'].apply(lambda x: 'no firstname' if len(str(x)) == 0 else str(x).split(',')[-1].strip()+ ' ')
df_pure['fullname'] = df_pure['firstname'] + df_pure['lastname']
df_pure.fillna('',inplace=True)

c:\Users\torbj\.conda\envs\newenv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
df_pure.set_index('fullname',inplace=True)

In [3]:
#we scan scopus, orcid and openAlex for Author IDs
orcid_list = []
orcid_list2 = []
orcid_list3 = []

scopusid_list = []
scopusid_list2 = []

openalexid_list = []
openalexid_list2 = []
openalexid_list3 = []

for i in tqdm(df_pure.index):

    first_name = df_pure['firstname'][i]
    last_name = df_pure['lastname'][i]
    full_name = df_pure['fullname'][i]

    #orcid (by name and affiliation)
    orcid_name_search = requests.get(url=f'https://pub.orcid.org/v3.0/search/?q=family-name:{last_name}+AND+given-names:{first_name}+AND+current-institution-affiliation-name:{'Utrecht'}',
                               headers={'Accept':'application/json', 'Authorization': 'Bearer %s' % access_token}
                               )
    if orcid_name_search.json()['result'] == None:
        orcid = ''
    else:
        orcid = orcid_name_search.json()['result'][0]['orcid-identifier']['path']
    orcid_list.append(orcid)


    #scopus (by orcid)
    scopus_orcid_seach = requests.get(url=f'http://api.elsevier.com/content/search/author?query=ORCID({orcid})', 
                    headers={'X-ELS-APIKey':client_scopus['apikey'], 'X-ELS-Insttoken':client_scopus['insttoken'], 'accept':'application/json'},)
    try:
        scopusid = scopus_orcid_seach.json()['search-results']['entry'][0]['dc:identifier'][10:]
    except:
        scopusid = ''
    scopusid_list.append(scopusid)


    #scopus (by name and affiliation)
    scopus_name_search = requests.get(url=f'http://api.elsevier.com/content/search/author?query=AUTHLASTNAME({last_name})+AND+AUTHFIRST({first_name})+AND+AFFIL(Utrecht)', 
                headers={'X-ELS-APIKey':client_scopus['apikey'], 'X-ELS-Insttoken':client_scopus['insttoken'], 'accept':'application/json'},)
    
    #try to get scopusid
    try:
        scopusid2 = scopus_name_search.json()['search-results']['entry'][0]['dc:identifier'][10:]
    except:
        scopusid2 = ''
    scopusid_list2.append(scopusid2)

    #try to get orcid
    try:
        orcid2 = scopus_name_search.json()['search-results']['entry'][0]['orcid']
    except:
        orcid2 = ''
    orcid_list2.append(orcid2)


    # #openAlex (by orcid1)
    # try:
    #     openalex_orcid_search = pyalex.Authors()[f"https://orcid.org/{orcid}"]
    #     openalexid = openalex_orcid_search['id'][21:]

    # except:
    #     openalexid = ''
    # openalexid_list.append(openalexid)

    # #openAlex (by orcid2)
    # try:
    #     openalex_orcid2_search = pyalex.Authors()[f"https://orcid.org/{orcid2}"]
    #     openalexid2 = openalex_orcid2_search['id'][21:]
    # except:
    #     openalexid2 = ''
    # openalexid_list2.append(openalexid2)


    # #openAlex (by name and affiliation)
    # openalex_name_search = pyalex.Authors().search(full_name).filter(affiliations={'institution.id':"I193662353"}).get()
    # if len(openalex_name_search) > 0:
    #     try:
    #         orcid3 = openalex_name_search[0]['orcid'][19:]
    #     except:
    #         orcid3 = ''
    #     openalexid3 = openalex_name_search[0]['id'][21:]
    # else:
    #     orcid3 = ''
    #     openalexid3 = ''
    # orcid_list3.append(orcid3)
    # openalexid_list3.append(openalexid3)


  0%|          | 0/25376 [00:00<?, ?it/s]

  8%|▊         | 1974/25376 [1:20:54<15:59:15,  2.46s/it]


KeyError: 'result'

In [4]:
orcid_name_search.json()

{'response-code': 500,
 'developer-message': 'org.apache.solr.client.solrj.impl.HttpSolrClient.RemoteSolrException Full validation error: Error from server at http://localhost:7983/solr/profile: SolrCore is loading',
 'user-message': 'Something went wrong in ORCID.',
 'error-code': 9008,
 'more-info': 'https://members.orcid.org/api/resources/troubleshooting'}

In [12]:
def join_func2(x,y):
    if x=='':
        x=y
    return x

def join_func3(x,y,z):
    if x=='':
        x=y
        if x=='':
            x=z
    return x

#join orcid lists
df_orcid = pd.DataFrame({'id1':orcid_list, 'id2':orcid_list2, 'id3':orcid_list3})
df_orcid['orcid'] = df_orcid.apply(lambda x: join_func3(x['id1'],x['id2'],x['id3']), axis=1)
orcid_list_final = df_orcid['orcid'].to_list()


#join openalexid lists
df_openalexid = pd.DataFrame({'id1':openalexid_list, 'id2':openalexid_list2, 'id3':openalexid_list3})
df_openalexid['openalexid'] = df_openalexid.apply(lambda x: join_func3(x['id1'],x['id2'],x['id3']), axis=1)
openalexid_list_final = df_openalexid['openalexid'].to_list()


#join scopusid lists
df_scopus = pd.DataFrame({'id1':scopusid_list, 'id2':scopusid_list2})
df_scopus['scopusid'] = df_scopus.apply(lambda x: join_func2(x['id1'],x['id2']), axis=1)
scopusid_list_final = df_scopus['scopusid'].to_list()

df_names['orcID'] = orcid_list_final
df_names['openAlexID'] = openalexid_list_final
df_names['scopusID'] = scopusid_list_final

df_names.to_csv('people_in_S4S_pureFiltered_withAuthorIDs.csv',index_label='Index')


old stuff below

In [2]:
#? (1) search orcid by name & affiliation
orcid_list = {}
scopus_id_list = {}

for i in tqdm(df_persons.index):
    first_name = df_persons['firstname'][i] #1st search term
    last_name = df_persons['lastname'][i] #1st search term


    name_search = requests.get(url=f'https://pub.orcid.org/v3.0/search/?q=family-name:{last_name}+AND+given-names:{first_name}',
                               headers={'Accept':'application/json', 'Authorization': 'Bearer %s' % access_token}
                               )

    if name_search.json()['result'] != None:
        found = False
        j = 0
        while found == False:
            orcid = name_search.json()['result'][j]['orcid-identifier']['path']
            orcid_record = requests.get(url=f'https://pub.orcid.org/v3.0/{orcid}/record',
                                        headers={'Accept':'application/json', 'Authorization': 'Bearer %s' % access_token})
            
            orcid_firstname = orcid_record.json()['person']['name']['given-names']['value']
            orcid_lastname = orcid_record.json()['person']['name']['family-name']['value']
            orcid_affils = []
            for k in range(len(orcid_record.json()['activities-summary']['employments']['affiliation-group'])):
                orcid_affils.append(orcid_record.json()['activities-summary']['employments']['affiliation-group'][k]['summaries'][0]['employment-summary']['organization']['name'])
                if orcid_affils[k] == 'Utrecht University' or orcid_affils[k] == 'Universiteit Utrecht':
                     uu_index = k
            if (('Universiteit Utrecht' in orcid_affils) or ('Utrecht University' in orcid_affils)) and (orcid_firstname == first_name) and (orcid_lastname == last_name):
                orcid_list[i] = orcid
                found = True

            else:
                 j+=1
                 if j > 5 or j >= len(name_search.json()['result']):
                      orcid_list[i] = ''
                      break
                 
    else:
            orcid_list[i] = ''


df_persons['orcid'] = orcid_list
df_persons.to_csv('person_names.csv',index_label='Index')


  2%|▏         | 6/272 [00:22<16:20,  3.69s/it]


KeyboardInterrupt: 

In [3]:
# (2) search scopus by orcid
scopus_id_list = {}

for i in tqdm(df_persons.index):
    if df_persons['orcid'][i] != '':
        orcid = df_persons['orcid'][i]
        resp = requests.get(url=f'http://api.elsevier.com/content/search/author?query=ORCID({orcid})', 
                    headers={'X-ELS-APIKey':client_scopus['apikey'], 'X-ELS-Insttoken':client_scopus['insttoken'], 'accept':'application/json'},)
    
        try:
            scopus_id = resp.json()['search-results']['entry'][0]['dc:identifier'][10:]
            scopus_id_list[i] = scopus_id
        except:
            scopus_id_list[i] = ''
    else:
        scopus_id_list[i] = ''
        
df_persons['scopusId'] = scopus_id_list

100%|██████████| 272/272 [01:02<00:00,  4.33it/s]


In [4]:
# (3) search scopus by name and affiliation
scopus_id_list = {}
orcid_list = {}

for i in tqdm(df_persons.index):
    # if df_persons['scopusId'][i] == '':
        first_name = df_persons['firstname'][i]

        #initial search? -> not sufficient
        # first_name = df_persons['firstname'][i][0].upper()+'.'
        
        last_name = df_persons['lastname'][i]
        resp = requests.get(url=f'http://api.elsevier.com/content/search/author?query=AUTHLASTNAME({last_name})+AND+AUTHFIRST({first_name})+AND+AFFIL(Utrecht)', 
                headers={'X-ELS-APIKey':client_scopus['apikey'], 'X-ELS-Insttoken':client_scopus['insttoken'], 'accept':'application/json'},)
        try:
            #saving the scopusid
            scopus_id = resp.json()['search-results']['entry'][0]['dc:identifier'][10:]
            scopus_id_list[i] = scopus_id
            # print(scopus_id)


        except:
            scopus_id_list[i] = ''
            orcid_list[i] = ''


        #see if there is an orcid associated
        try:
            orcid =  resp.json()['search-results']['entry'][0]['orcid']
            orcid_list[i] = orcid
            # print(orcid)
        except:
            orcid_list[i] = ''

    # else:
        # scopus_id_list[i] = df_persons['scopusId'][i]
        # orcid_list[i] = df_persons['orcid'][i]
        
df_persons['scopusId'] = scopus_id_list
df_persons['orcid'] = orcid_list

100%|██████████| 272/272 [03:52<00:00,  1.17it/s]


In [5]:
df_persons[df_persons['scopusId'] != '']

,firstname,lastname,Role,fullname,faculty,department,orcid,scopusId
Index,,,,,,,,
1,Ahmed,Al-Mayahi,PhD Candidate,Ahmed Al-Mayahi,NaN,NaN,0000-0002-8922-2165,57205546261
3,Isabel,Arends,Dean,Isabel Arends,beta,"Science, Dean",0000-0002-1015-4682,7004471681
4,Charissa,de Bekker,Assistant Professor,Charissa de Bekker,NaN,NaN,0000-0002-7325-172X,23099000200
5,Roeland,Berendsen,Researcher,Roeland Berendsen,NaN,NaN,,36237571800
6,Hugo,de Boer,Associate Professor,Hugo de Boer,NaN,NaN,,35147701900
...,...,...,...,...,...,...,...,...
256,Frances,Dunn,NaN,Frances Dunn,NaN,NaN,0000-0003-3726-7158,56818633500
260,Liping,Dai,NaN,Liping Dai,REBO,NaN,,55683149200
262,Katinka,Wijsman,NaN,Katinka Wijsman,NaN,NaN,,55531558400


In [5]:
# (5) last resort for now: search scopus by name
for i in tqdm(df_persons.index):
    if df_persons['scopusId'][i] == '':
        first_name = df_persons['firstname'][i]
        last_name = df_persons['lastname'][i]
        resp = requests.get(url=f'http://api.elsevier.com/content/search/author?query=AUTHLASTNAME({last_name})+AND+AUTHFIRST({first_name})', 
                headers={'X-ELS-APIKey':client_scopus['apikey'], 'X-ELS-Insttoken':client_scopus['insttoken'], 'accept':'application/json'},)
        try:
            #saving the scopusid
            scopus_id = resp.json()['search-results']['entry'][0]['dc:identifier'][10:]
            scopus_id_list[i] = scopus_id
            # print(scopus_id)


        except:
            scopus_id_list[i] = ''
            orcid_list[i] = ''


        #see if there is an orcid associated
        try:
            orcid =  resp.json()['search-results']['entry'][0]['orcid']
            orcid_list[i] = orcid
            # print(orcid)
        except:
            orcid_list[i] = ''

    else:
        scopus_id_list[i] = df_persons['scopusId'][i]
        orcid_list[i] = df_persons['orcid'][i]
        
df_persons['scopusId'] = scopus_id_list
df_persons['orcid'] = orcid_list

100%|██████████| 272/272 [01:38<00:00,  2.77it/s]


In [31]:
df_persons.to_csv('person_names.csv')